In [ ]:
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs
import string

import sys
sys.path.append('../')

from plotfunctions import add_lonlat

np.seterr(all='ignore')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
startyear = 2003
endyear   = 2008

In [ ]:
proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0)#260-360)
axex = [245.8,251,-75.2,-74.4]
#axex2 = [245.7,246.1,-75.11,-74.85]
axex2 = [246.8,247,-74.6,-74.3]

cmap1 = mpl.colors.ListedColormap(plt.get_cmap('cmo.deep_r')(np.linspace(0,.4,255)))
cmap2 = 'cmo.rain'
cmap3 = mpl.colors.ListedColormap(plt.get_cmap('cmo.gray')(np.linspace(.2,.6,255)))

x0,x1,y0,y1 = 3400,3800,7700,8100

In [ ]:
vmax = 100
vmin = -10
linthresh=1
linscale=.2

vmax = 100
vmin = -10
linthresh = .3
linscale = .25


fracpos = (np.log10(vmax/linthresh)+linscale)/(np.log10(vmax/linthresh)+np.log10(-(vmin/linthresh))+2*linscale)
#print(fracpos)
nneg = np.int_((1-fracpos)*256)

colors1 = plt.get_cmap('cmo.ice_r')(np.linspace(0,.9,nneg))
colors2 = plt.cm.inferno(np.linspace(0.1, 1, 256-nneg))

# combine them and build a new colormap
colors = np.vstack((colors1, colors2))
mymap = mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors)


In [ ]:
def makebackground(ax,axx):
    cmap = plt.get_cmap('ocean')
    ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(x0,x1),y=slice(y0,y1))
    mask = xr.where(ds.mask==1,2,ds.mask)
    ds = add_lonlat(ds)
    
    ax.set_facecolor(".7")
    
    im1 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==0,ds.bed,np.nan),cmap=cmap1,shading='nearest',transform=ccrs.PlateCarree(),vmin=-1500,vmax=0)
    im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==2,ds.bed,np.nan),cmap=cmap3,shading='nearest',transform=ccrs.PlateCarree(),vmin=-2000,vmax=0)

    ax.annotate('Dotson Channel',(.65,.65),xytext=(.65,.8),va='bottom',ha='center',arrowprops=dict(arrowstyle="->"),xycoords='axes fraction')
    #ax.annotate('melt\nmaximum',(.4,.47),xytext=(.5,.55),va='center',ha='left',arrowprops=dict(arrowstyle="->"),xycoords='axes fraction')
    ax.annotate('coastal channel',(.45,.25),xytext=(.35,.15),va='center',ha='right',arrowprops=dict(arrowstyle="->"),xycoords='axes fraction')

    #ax.annotate('transverse\nchannels',(.75,.5),xytext=(.85,.35),va='top',ha='center',arrowprops=dict(arrowstyle="->"),xycoords='axes fraction')
    #ax.annotate(' ',(.95,.55),xytext=(.85,.35),va='top',ha='center',arrowprops=dict(arrowstyle="->"),xycoords='axes fraction')

    
    ax.set_extent(axx,crs=ccrs.PlateCarree())

In [ ]:
def plotmelt(ax,axx,lon,lat,melt,title=None):
    makebackground(ax,axx)
    ax.set_extent(axx,crs=ccrs.PlateCarree())
    im = ax.pcolormesh(lon,lat,melt,cmap=mymap,norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale),shading='nearest',transform=ccrs.PlateCarree())

    ax.set_title(title,loc='left',fontsize=10)
        
    return im

In [ ]:
plt.style.use('style_paper')

mpl.rc('figure.subplot',left=.01,right=.98,top=.95,bottom=.15,wspace=.05,hspace=.2)

fig = plt.figure(figsize=(7,4.5))

#Geometry
ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
ds = ds.isel(x=slice(x0,x1),y=slice(y0,y1))
ds = add_lonlat(ds)

ax = fig.add_subplot(221,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))
makebackground(ax,axex2)
ax.set_extent(axex2,crs=ccrs.PlateCarree())
ax.set_title('a) Geometry',loc='left',fontsize=10)
im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(ds.mask==3,ds.thickness,np.nan),cmap=cmap2,shading='nearest',transform=ccrs.PlateCarree(),vmin=0,vmax=600)

y0,y1 = 7950,8060
for xx in [3522,3536,3560]:
    ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(xx,xx+1),y=slice(y0,y1))
    ds = add_lonlat(ds)
    ax.plot(ds.lon,ds.lat,c='k',transform=ccrs.PlateCarree(),zorder=10)

ax.text(.2,.9,'C',transform=ax.transAxes)
ax.text(.4,.9,'B',transform=ax.transAxes)
ax.text(.75,.9,'A',transform=ax.transAxes)


#Obs
ds = xr.open_dataset('../../../data/gourmelen/CrossonDotson.nc')
ds = add_lonlat(ds)
melt = ds.Band1

ax = fig.add_subplot(222,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))
im = plotmelt(ax,axex2,ds.lon,ds.lat,melt,'b) Observations')

y0,y1 = 7950,8060
for xx in [3522,3536,3560]:
    ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(xx,xx+1),y=slice(y0,y1))
    ds = add_lonlat(ds)
    ax.plot(ds.lon,ds.lat,c='k',transform=ccrs.PlateCarree(),zorder=10)

#MITgcm
timep= slice(f"{startyear}-1-1",f"{endyear}-12-31")
ds = xr.open_dataset('../../../data/paulholland/melt.nc')
ds = ds.sel(LONGITUDE=slice(360-114.7,360-109),LATITUDE=slice(-75.3,-74.1),TIME=timep)
ds = ds.mean(dim='TIME')
lon   = (ds.LONGITUDE-360.).values
lat   = (ds.LATITUDE-.05).values
melt = ds.melt.values

ax = fig.add_subplot(223,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))
im = plotmelt(ax,axex2,lon,lat,np.where(melt==0,np.nan,melt),'c) 3D model')

#LADDIE
#ds = xr.open_dataset(f'../../results/spinup/CrossDots_0.5_mitgcm_{startyear}_{endyear}_100.nc')
ds = xr.open_dataset(f'../../results/spinup/CrossDots_0.5_tanh_Tdeep0.5_ztcl-500_050.nc')

ax = fig.add_subplot(224,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))
im = plotmelt(ax,axex2,ds.lon,ds.lat,np.where(ds.tmask,ds.melt,np.nan),'d) LADDIE')

y0,y1 = 7950,8060
for xx in [3522,3536,3560]:
    ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(xx,xx+1),y=slice(y0,y1))
    ds = add_lonlat(ds)
    ax.plot(ds.lon,ds.lat,c='k',transform=ccrs.PlateCarree(),zorder=10)

#Colorbars

ax = fig.add_subplot(111)
ax.set_visible(False)
cax = inset_axes(ax,width="44%",height="5%",loc='lower left',bbox_to_anchor=(0.025,-.08,1,1),bbox_transform=ax.transAxes,borderpad=0)
cb = plt.colorbar(im2,cax=cax,orientation='horizontal')
cb.set_label('Ice shelf thickness [m]')

cax = inset_axes(ax,width="44%",height="5%",loc='lower left',bbox_to_anchor=(.54,-.08,1,1),bbox_transform=ax.transAxes,borderpad=0)
ticks = [-10,-3,-1,-.3,0,.3,1,3,10,30,100]
cb = plt.colorbar(im,cax=cax,orientation='horizontal',extend='both')
cb.set_ticks(ticks)
cb.set_ticklabels(ticks)
cb.set_label('Freezing / Melt rate [m yr$^{-1}$]')

plt.savefig('../../figures/draftplot_CD_DC.png',dpi=450)